# SDG Meter BERT algorithm

Algorithm's source & explanations :

(1) https://www.kaggle.com/javaidnabi/toxic-comment-classification-using-bert/output#Use-the-InputExample-class-from-BERT's-run_classifier-code-to-create-examples-from-the-data

(2) https://towardsdatascience.com/building-a-multi-label-text-classifier-using-bert-and-tensorflow-f188e0ecdc5d

(3)https://easyai.tech/en/blog/multi-label-text-classification-using-bert-the-mighty-transformer/

### Import useful Libraries

In [1]:
# It is important to have this exact version for our algorithm
!pip install tensorflow==1.15.0

     |████████████████████████████████| 124.0 MB 2.9 MB/s eta 0:00:01     |████████████████▉               | 65.5 MB 33.5 MB/s eta 0:00:02
     |████████████████████████████████| 104 kB 12.4 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 6.8 MB/s  eta 0:00:01
     |████████████████████████████████| 65 kB 11.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 20.5 MB/s eta 0:00:01
     |████████████████████████████████| 503 kB 33.9 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 4.8 MB/s  eta 0:00:01
     |████████████████████████████████| 50 kB 16.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 35.2 MB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 44.4 MB/s eta 0:00:01
     |████████████████████████████████| 88 kB 18.7 MB/s eta 0:00:01
  Created wheel for absl-py: filename=absl_py-0.9.0-py3-none-any.whl size=121931 sha256=b815df1e643c755ae19b70af1850e8ee3d8f5a951363fac40784de1bd5f6da42
  Stored in directory: 

In [2]:
print(tf.__version__)

1.15.0


In [ ]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)

import os
import collections

from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
import random

In [5]:
!pip install bert-tensorflow

     |████████████████████████████████| 67 kB 3.5 MB/s eta 0:00:011


In [2]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling

In [3]:
# /!\
# Here we first need to download the following link file and save it in the same directory: 
# https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
# More information about this file in link (2) 

#import tokenization
#import modeling
BERT_VOCAB= 'uncased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = 'uncased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = 'uncased_L-12_H-768_A-12/bert_config.json'

In [4]:
tokenization.validate_case_matches_checkpoint(True,BERT_INIT_CHKPNT)
tokenizer = tokenization.FullTokenizer(
      vocab_file=BERT_VOCAB, do_lower_case=True)

In [169]:
# Import "res.csv" Database
# In order to better understand our database composition you can check the " Database_conception_BERT" jupyter notebook
res = pd.read_csv('res.csv',sep=';')

In [170]:
# Make sure that the column "Text" is string (str) format
res['Text'] = res['Text'].astype(str)

In [171]:
# Replace empty cells by 0
res = res.fillna(0) 

# Shuffle order to mix SDGs numbers
random.seed(30)
res = res.sample(frac=1)

In [172]:
# Make sure all other columns are numeric
cols=[i for i in res.columns if i not in ["Text"]]
for col in cols:
    res[col]=pd.to_numeric(res[col], downcast='integer')

In [173]:
# Replace \n by white space 
res = res.replace(r'\n',  ' ', regex=True)

In [174]:
# See extract of the dataframe res
res

,ID,Text,SDG_1,SDG_2,SDG_3,SDG_4,SDG_5,SDG_6,SDG_7,SDG_8,SDG_9,SDG_10,SDG_11,SDG_12,SDG_13,SDG_14,SDG_15,SDG_16,SDG_17_
4054,4054,This paper investigates the 10 highest-burden ...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1125,1125,The likelihood of increased variability and ex...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2152,2152,12.1 Implement the 10-year framework of progra...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
878,878,9 Governments should be needed to labour marke...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
623,623,The association between gender and the warmest...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4333,4333,Monitoring of food prices is provided by publi...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2399,2399,Mexico is committed to the promotion of develo...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5518,5518,"In 2015, but it is aligned with productive cap...",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2944,2944,"This guidance document, entitled “Responsible ...",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [175]:
# Split New_text in train and test (80/20) sample (Sklearn function)
train, test = train_test_split(res, test_size=0.2)

In [176]:
# Specify columns order
train = train[['ID','Text','SDG_1','SDG_2','SDG_3','SDG_4','SDG_5',
               'SDG_6','SDG_7', 'SDG_8','SDG_9','SDG_10',
               'SDG_11','SDG_12', 'SDG_13','SDG_14','SDG_15','SDG_16','SDG_17_']] 

In [178]:
# In the prediction, if you dont want to see text belonging : Only keep ID and Text for test data
#test = test[['ID','Text']] 

In [180]:
# If you want to test the model by adding a text providing from an other source by text file :  : 
Entry = pd.read_csv("input.txt", sep = "\t", header=None)

# Or by manual entry : 
#Entry = ["In signing Agenda 2030, Governments around the world committed to ending poverty in all its manifestations, including its most extreme forms, over the next 15 years. They resolved that all people everywhere should enjoy a basic standard of living. This includes social protection benefits for the poor and most vulnerable and ensuring that people harmed by conflict and natural hazards receive adequate support, including access to basic services."]

In [181]:
Entry

,0
0,Remdesivir is an antiviral drug that was origi...


In [182]:
# Take here entry text to add at the end of test dataframe :
test = test.append({'ID' : 7000 , 'Text' : Entry[0], 'SDG_1' : 1,'SDG_2': 0,'SDG_3': 0,'SDG_4': 0,'SDG_5': 0,
                    'SDG_6': 0,'SDG_7': 0, 'SDG_8': 0,'SDG_9': 0,'SDG_10': 0,'SDG_11': 0,'SDG_12': 0, 
                    'SDG_13': 0,'SDG_14': 0,'SDG_15': 0,'SDG_16': 0,'SDG_17_': 0 } , ignore_index=True)

In [183]:
test

,ID,Text,SDG_1,SDG_2,SDG_3,SDG_4,SDG_5,SDG_6,SDG_7,SDG_8,SDG_9,SDG_10,SDG_11,SDG_12,SDG_13,SDG_14,SDG_15,SDG_16,SDG_17_
0,2037,Ending hunger is threatened or salinized in ru...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1132,Historically low productivity and economic gro...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,5137,The difference between the approved and the im...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,4330,1 This makers with available . 7 The 2030 Agen...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,2025,"Groundwater of extreme weather events, threate...",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,5521,This then focuses on inefficient and to energy...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1201,5675,"4.5 By 2030, ensure the empowerment of is a ce...",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1202,2718,This paper highlights the heat generated on in...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1203,4821,5.3 Eliminate all women aged 15 to provide wom...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [184]:
# Make sure that the column "Text" is string (str) format
test['Text'] = test['Text'].astype(str)

## BERT

In [185]:
ID = 'ID'
DATA_COLUMN = 'Text'
LABEL_COLUMNS = ['SDG_1','SDG_2','SDG_3','SDG_4','SDG_5','SDG_6','SDG_7',
                 'SDG_8','SDG_9','SDG_10','SDG_11','SDG_12', 'SDG_13','SDG_14',
                 'SDG_15','SDG_16','SDG_17_']


In [186]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, labels=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            labels: (Optional) [string]. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.labels = labels


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_ids, is_real_example=True):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids,
        self.is_real_example=is_real_example

In [187]:
def create_examples(df, labels_available=True):
    """Creates examples for the training and dev sets."""
    examples = []
    for (i, row) in enumerate(df.values):
        guid = row[0]
        text_a = row[1]
        if labels_available:
            labels = row[2:]
        else:
            labels = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
        examples.append(
            InputExample(guid=guid, text_a=text_a, labels=labels))
    return examples

In [188]:
TRAIN_VAL_RATIO = 0.90
LEN = train.shape[0]
SIZE_TRAIN = int(TRAIN_VAL_RATIO*LEN)

x_train = train[:SIZE_TRAIN]
x_val = train[SIZE_TRAIN:]

# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_examples = create_examples(x_train)

In [189]:
train.shape, x_train.shape, x_val.shape

((4812, 19), (4330, 19), (482, 19))

In [190]:
import pandas

def convert_examples_to_features(examples,  max_seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s."""

    features = []
    for (ex_index, example) in enumerate(examples):
        print(example.text_a)
        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)
            
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length - 2)]

        
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

    
        input_mask = [1] * len(input_ids)

      
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        
        labels_ids = []
        for label in example.labels:
            labels_ids.append(int(label))

        if ex_index < 0:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("label: %s (id = %s)" % (example.labels, labels_ids))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_ids=labels_ids))
    return features

In [191]:
MAX_SEQ_LENGTH = 168

In [192]:
def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels, use_one_hot_embeddings):
    """Creates a classification model."""
    model = modeling.BertModel(
        config=bert_config,
        is_training=is_training,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=segment_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)

    # For using the token-level output, use model.get_sequence_output() instead. 
    output_layer = model.get_pooled_output()

    hidden_size = output_layer.shape[-1].value

    output_weights = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable(
        "output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):
        if is_training:
           
            output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        
        # probabilities = tf.nn.softmax(logits, axis=-1) ## multiclass Classification 
        probabilities = tf.nn.sigmoid(logits)### multi-label Classification
        
        labels = tf.cast(labels, tf.float32)
        tf.logging.info("num_labels:{};logits:{};labels:{}".format(num_labels, logits, labels))
        per_example_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)
        loss = tf.reduce_mean(per_example_loss)

        return (loss, per_example_loss, logits, probabilities)


def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
    """Returns `model_fn` closure for TPUEstimator."""

    def model_fn(features, labels, mode, params):  
        """The `model_fn` for TPUEstimator."""

        
        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]
        is_real_example = None
        if "is_real_example" in features:
             is_real_example = tf.cast(features["is_real_example"], dtype=tf.float32)
        else:
             is_real_example = tf.ones(tf.shape(label_ids), dtype=tf.float32)

        is_training = (mode == tf.estimator.ModeKeys.TRAIN)

        (total_loss, per_example_loss, logits, probabilities) = create_model(
            bert_config, is_training, input_ids, input_mask, segment_ids, label_ids,
            num_labels, use_one_hot_embeddings)

        tvars = tf.trainable_variables()
        initialized_variable_names = {}
        scaffold_fn = None
        if init_checkpoint:
            (assignment_map, initialized_variable_names
             ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
            if use_tpu:

                def tpu_scaffold():
                    tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
                    return tf.train.Scaffold()

                scaffold_fn = tpu_scaffold
            else:
                tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

        tf.logging.info("**** Trainable Variables ****")
        for var in tvars:
            init_string = ""
            if var.name in initialized_variable_names:
                init_string = ", *INIT_FROM_CKPT*"
            

        output_spec = None
        if mode == tf.estimator.ModeKeys.TRAIN:

            train_op = optimization.create_optimizer(
                total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                train_op=train_op,
                scaffold=scaffold_fn)
        elif mode == tf.estimator.ModeKeys.EVAL:

            def metric_fn(per_example_loss, label_ids, probabilities, is_real_example):

                logits_split = tf.split(probabilities, num_labels, axis=-1)
                label_ids_split = tf.split(label_ids, num_labels, axis=-1)
                # metrics change to auc of every class
                eval_dict = {}
                for j, logits in enumerate(logits_split):
                    label_id_ = tf.cast(label_ids_split[j], dtype=tf.int32)
                    current_auc, update_op_auc = tf.metrics.auc(label_id_, logits)
                    eval_dict[str(j)] = (current_auc, update_op_auc)
                eval_dict['eval_loss'] = tf.metrics.mean(values=per_example_loss)
                return eval_dict

               

            eval_metrics = metric_fn(per_example_loss, label_ids, probabilities, is_real_example)
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                eval_metric_ops=eval_metrics,
                scaffold=scaffold_fn)
        else:
            print("mode:", mode,"probabilities:", probabilities)
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                predictions={"probabilities": probabilities},
                scaffold=scaffold_fn)
        return output_spec

    return model_fn

In [193]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 1e-5
NUM_TRAIN_EPOCHS = 2.0

WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500

In [194]:
OUTPUT_DIR = "output"
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    keep_checkpoint_max=1,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [195]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_ids)

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    num_examples = len(features)

    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples, len(LABEL_COLUMNS)], dtype=tf.int32),
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn

In [196]:
class PaddingInputExample(object):
    """Fake example so the num input examples is a multiple of the batch size.
    When running eval/predict on the TPU, we need to pad the number of examples
    to be a multiple of the batch size, because the TPU requires a fixed batch
    size. The alternative is to drop the last batch, which is bad because it means
    the entire output data won't be generated.
    We use this class instead of `None` because treating `None` as padding
    battches could cause silent errors.
    """

In [197]:
def convert_single_example(ex_index, example, max_seq_length,
                           tokenizer):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    if isinstance(example, PaddingInputExample):
        return InputFeatures(
            input_ids=[0] * max_seq_length,
            input_mask=[0] * max_seq_length,
            segment_ids=[0] * max_seq_length,
            label_ids=0,
            is_real_example=False)

    tokens_a = tokenizer.tokenize(example.text_a)
    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)

    if tokens_b:
        
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[0:(max_seq_length - 2)]

    
    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    if tokens_b:
        for token in tokens_b:
            tokens.append(token)
            segment_ids.append(1)
        tokens.append("[SEP]")
        segment_ids.append(1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    
    input_mask = [1] * len(input_ids)

    
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    labels_ids = []
    for label in example.labels:
        labels_ids.append(int(label))


    feature = InputFeatures(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids,
        label_ids=labels_ids,
        is_real_example=True)
    return feature


def file_based_convert_examples_to_features(
        examples, max_seq_length, tokenizer, output_file):
    """Convert a set of `InputExample`s to a TFRecord file."""

    writer = tf.python_io.TFRecordWriter(output_file)

    for (ex_index, example) in enumerate(examples):
        
        feature = convert_single_example(ex_index, example,
                                         max_seq_length, tokenizer)

        def create_int_feature(values):
            f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
            return f

        features = collections.OrderedDict()
        features["input_ids"] = create_int_feature(feature.input_ids)
        features["input_mask"] = create_int_feature(feature.input_mask)
        features["segment_ids"] = create_int_feature(feature.segment_ids)
        features["is_real_example"] = create_int_feature(
            [int(feature.is_real_example)])
        if isinstance(feature.label_ids, list):
            label_ids = feature.label_ids
        else:
            label_ids = feature.label_ids[0]
        features["label_ids"] = create_int_feature(label_ids)

        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())
    writer.close()


def file_based_input_fn_builder(input_file, seq_length, is_training,
                                drop_remainder):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""

    name_to_features = {
        "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
        "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "label_ids": tf.FixedLenFeature([17], tf.int64),
        "is_real_example": tf.FixedLenFeature([], tf.int64),
    }

    def _decode_record(record, name_to_features):
        """Decodes a record to a TensorFlow example."""
        example = tf.parse_single_example(record, name_to_features)

        # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
        # So cast all int64 to int32.
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.to_int32(t)
            example[name] = t

        return example

    def input_fn(params):
        """The actual input function."""
        batch_size = params["batch_size"]

        # For training, we want a lot of parallel reading and shuffling.
        # For eval, we want no shuffling and parallel reading doesn't matter.
        d = tf.data.TFRecordDataset(input_file)
        if is_training:
            d = d.repeat()
            d = d.shuffle(buffer_size=100)

        d = d.apply(
            tf.contrib.data.map_and_batch(
                lambda record: _decode_record(record, name_to_features),
                batch_size=batch_size,
                drop_remainder=drop_remainder))

        return d

    return input_fn


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [198]:
#from pathlib import Path
train_file = os.path.join('../One-Planet', "train.tf_record")
#filename = Path(train_file)
if not os.path.exists(train_file):
    open(train_file, 'w').close()

### Create an input function for training

In [199]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_examples) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [200]:
file_based_convert_examples_to_features(
            train_examples, MAX_SEQ_LENGTH, tokenizer, train_file)
tf.logging.info("***** Running training *****")
tf.logging.info("  Num examples = %d", len(train_examples))
tf.logging.info("  Batch size = %d", BATCH_SIZE)
tf.logging.info("  Num steps = %d", num_train_steps)

INFO:tensorflow:***** Running training *****
INFO:tensorflow:  Num examples = 4330
INFO:tensorflow:  Batch size = 32
INFO:tensorflow:  Num steps = 270


In [201]:
train_input_fn = file_based_input_fn_builder(
    input_file=train_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True) 

In [202]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)
model_fn = model_fn_builder(
  bert_config=bert_config,
  num_labels= len(LABEL_COLUMNS),
  init_checkpoint=BERT_INIT_CHKPNT,
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=False,
  use_one_hot_embeddings=False)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': 'output', '_tf_random_seed': None, '_save_summary_steps': 500, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a40533710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [203]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Skipping training since max_steps has already saved.
Training took time  0:00:00.008756


In [204]:
eval_file = os.path.join('../One-Planet', "eval.tf_record")
#filename = Path(train_file)
if not os.path.exists(eval_file):
    open(eval_file, 'w').close()

eval_examples = create_examples(x_val)
file_based_convert_examples_to_features(
    eval_examples, MAX_SEQ_LENGTH, tokenizer, eval_file)

In [205]:
# This tells the estimator to run through the entire set.
eval_steps = None

eval_drop_remainder = False
eval_input_fn = file_based_input_fn_builder(
    input_file=eval_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:num_labels:17;logits:Tensor("loss/BiasAdd:0", shape=(?, 17), dtype=float32);labels:Tensor("loss/Cast:0", shape=(?, 17), dtype=float32)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-07-29T15:46:30Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from output/model.ckpt-270
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-07-29-15:48:00
INFO:tensorflow:Saving dict for global step 270: 0 = 0.93348897, 1 = 0.90645283, 10 = 0.96061075, 11 = 0.97353923, 12 = 0.89795995, 13 = 0.9410124, 14 = 0.9667804, 15 = 0.86605024, 16 = 0.8563676, 2 = 0.96656173, 3 = 0.9414221, 4 = 0.9608739, 5 = 0.92037785, 6 = 0.857257, 7 = 0.8032385, 8 = 0.8125348, 9 = 0.90743697, eval_loss = 0.19654994, global_step = 270, loss = 0.19672829
INFO:tensorflow:Saving 'checkpoint_path' summar

### Use the InputExample class from BERT's run_classifier code to create examples from the data

In [206]:
output_eval_file = os.path.join("../One-Planet", "eval_results.txt")
with tf.gfile.GFile(output_eval_file, "w") as writer:
    tf.logging.info("***** Eval results *****")
    for key in sorted(result.keys()):
        tf.logging.info("  %s = %s", key, str(result[key]))
        writer.write("%s = %s\n" % (key, str(result[key])))

INFO:tensorflow:***** Eval results *****
INFO:tensorflow:  0 = 0.93348897
INFO:tensorflow:  1 = 0.90645283
INFO:tensorflow:  10 = 0.96061075
INFO:tensorflow:  11 = 0.97353923
INFO:tensorflow:  12 = 0.89795995
INFO:tensorflow:  13 = 0.9410124
INFO:tensorflow:  14 = 0.9667804
INFO:tensorflow:  15 = 0.86605024
INFO:tensorflow:  16 = 0.8563676
INFO:tensorflow:  2 = 0.96656173
INFO:tensorflow:  3 = 0.9414221
INFO:tensorflow:  4 = 0.9608739
INFO:tensorflow:  5 = 0.92037785
INFO:tensorflow:  6 = 0.857257
INFO:tensorflow:  7 = 0.8032385
INFO:tensorflow:  8 = 0.8125348
INFO:tensorflow:  9 = 0.90743697
INFO:tensorflow:  eval_loss = 0.19654994
INFO:tensorflow:  global_step = 270
INFO:tensorflow:  loss = 0.19672829


In [207]:
x_test = test
x_test = x_test.reset_index(drop=True)

test_file = os.path.join('../One-Planet', "test.tf_record")

if not os.path.exists(test_file):
    open(test_file, 'w').close()

test_examples = create_examples(x_test, False)
file_based_convert_examples_to_features(
    test_examples, MAX_SEQ_LENGTH, tokenizer, test_file)

In [208]:
predict_input_fn = file_based_input_fn_builder(
    input_file=test_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [209]:
print('Begin predictions!')
current_time = datetime.now()
predictions = estimator.predict(predict_input_fn)
print("Predicting took time ", datetime.now() - current_time)

Begin predictions!
Predicting took time  0:00:00.000153


In [210]:
def create_output(predictions):
    probabilities = []
    for (i, prediction) in enumerate(predictions):
        preds = prediction["probabilities"]
        probabilities.append(preds)
    dff = pd.DataFrame(probabilities)
    dff.columns = LABEL_COLUMNS
    
    return dff

In [211]:
output_df = create_output(predictions)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:num_labels:17;logits:Tensor("loss/BiasAdd:0", shape=(?, 17), dtype=float32);labels:Tensor("loss/Cast:0", shape=(?, 17), dtype=float32)
INFO:tensorflow:**** Trainable Variables ****
mode: infer probabilities: Tensor("loss/Sigmoid:0", shape=(?, 17), dtype=float32)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from output/model.ckpt-270
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [212]:
merged_df =  pd.concat([x_test, output_df], axis=1)
submission = merged_df.drop(['Text'], axis=1)
submission.to_csv("sample_submission.csv", index=False)

In [ ]:
# If you want to see all the results of the test phase + your added text :
#submission

In [213]:
# If you only want to see the result for your added text : 
# Extract last line of submission file 
submission_extract =  submission.tail(1)

# Conserve only last 17 columns 
submission_extract = submission_extract.iloc[:,-17:]

In [214]:
submission_extract

,SDG_1,SDG_2,SDG_3,SDG_4,SDG_5,SDG_6,SDG_7,SDG_8,SDG_9,SDG_10,SDG_11,SDG_12,SDG_13,SDG_14,SDG_15,SDG_16,SDG_17_
1204,0.088909,0.086221,0.111748,0.077155,0.078874,0.065481,0.063279,0.060621,0.068794,0.078501,0.080517,0.066355,0.090842,0.08054,0.057513,0.099257,0.08983


In [215]:
# Convert this dataframe to txt file in your repo : 
np.savetxt(r'submission.txt', submission_extract.values, fmt='%f')